### Table of Contents



# Import Modules

In [1]:
import ee, geemap, time

Intialize Geemap

In [2]:
Map = geemap.Map(center=(10.41, 37.62), zoom=11)

jedeb = ee.FeatureCollection("projects/ee-yilkalg3/assets/Jedeb_Watershed")
#Map.addLayer(jedeb, {}, 'Jedeb Watershed')

Set Dates

In [3]:
start = '2021-12-01'
end = '2022-04-30'
season = ee.Filter.date(start,end)

# Sentinel 2

In [4]:
# Load Median Sentinel 2 Reflectance 
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

# Filter the first Sentinel 2 image of the month
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR')\
    .filterBounds(jedeb)\
    .filter(season)\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))\
    .map(maskS2clouds)
 
# Median image
s2median = ee.Image(sentinel2.median()).clip(jedeb)
s2median.bandNames().getInfo()

visualization = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
}

Map.addLayer(s2median, visualization, 'RGB')

### Make Training Dataset

In [5]:
points = ee.FeatureCollection("projects/ee-yilkalg3/assets/TrainingJ")
print(points.size().getInfo())

896


### Train the classifier

In [6]:
# Use these bands for prediction (excluding red edge bands).
bands = ['B2', 'B3', 'B4', 'B5', 'B6','B7','B8', 'B8A', 'B11', 'B12']

# This property of the table stores the land cover labels.
label = 'Cover'

# Overlay the points on the imagery to get training.
sample = s2median.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 10}
)

In [7]:
classifier = ee.Classifier.libsvm(**{
  'kernelType': 'RBF',
  'gamma': 0.5,
  'cost': 128
}).train(sample, label, bands)

### Classify the Image

In [8]:
# Classify the image with the same bands used for training 
classified_RF1 = s2median.select(bands).classify(classifier)

### Accuracy assessment

#### Training

In [9]:
train_accuracy = classifier.confusionMatrix()
#train_accuracy.getInfo()

In [10]:
#train_accuracy.accuracy().getInfo()

In [11]:
#train_accuracy.kappa().getInfo()

In [12]:
#train_accuracy.producersAccuracy().getInfo()

In [13]:
#train_accuracy.consumersAccuracy().getInfo()

#### Validation 

In [14]:
validationGcp = ee.FeatureCollection("projects/ee-yilkalg3/assets/ValidationJ")
print(validationGcp.size().getInfo())

225


In [15]:
#validationGcp.first().getInfo()

In [16]:
# This property of the table stores the land cover labels.
label = 'Cover'

band = ['classification']
# Overlay the points on the imagery to get training.
test = classified_RF1.select(band).sampleRegions(
    **{'collection': validationGcp, 'properties':[label],'scale': 10}
)

In [22]:
test_accuracy = test.errorMatrix('Cover', 'classification')
test_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 3, 0, 0, 0, 0, 0, 0],
 [0, 1, 56, 0, 7, 6, 0, 0],
 [0, 0, 2, 20, 1, 11, 0, 0],
 [0, 0, 6, 0, 37, 0, 0, 1],
 [0, 0, 1, 1, 0, 49, 0, 2],
 [0, 0, 0, 0, 0, 0, 12, 0],
 [0, 0, 0, 0, 0, 2, 0, 7]]

In [18]:
test_accuracy.accuracy().getInfo()

0.8177777777777778

In [19]:
test_accuracy.kappa().getInfo()

0.7666388404037339

In [20]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [1],
 [0.8],
 [0.5882352941176471],
 [0.8409090909090909],
 [0.9245283018867925],
 [1],
 [0.7777777777777778]]

In [21]:
test_accuracy.consumersAccuracy().getInfo()

[[0,
  0.75,
  0.8615384615384616,
  0.9523809523809523,
  0.8222222222222222,
  0.7205882352941176,
  1,
  0.7]]

# Sentinel 1 and 2

## Sentinel 1 GRD

In [22]:
# Import Sentinel-1 collection
sentinel1 =  ee.ImageCollection('COPERNICUS/S1_GRD')\
                .filterBounds(jedeb)\
                .filter(season)\
                .filter(ee.Filter.eq('instrumentMode', 'IW'))\
                .filter(ee.Filter.eq('resolution_meters', 10)) 
                
# Number of tiles in the season
sentinel1 = sentinel1.sort('system:time_start')
sentinel1.size().getInfo()

13

In [23]:
# the acquisition times in the collection, formatted with Python's time module
acq_times = sentinel1.aggregate_array('system:time_start').getInfo()
[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]

['12/02/21',
 '12/14/21',
 '12/26/21',
 '01/07/22',
 '01/19/22',
 '01/31/22',
 '02/12/22',
 '02/24/22',
 '03/08/22',
 '03/20/22',
 '04/01/22',
 '04/13/22',
 '04/25/22']

In [24]:
# using monthly mean backscatter 

s1Dec = ee.Image(sentinel1.filterDate("2021-12-01","2021-12-31").mean()).select(["VV","VH"],["VVdec","VHdec"])
s1Jan = ee.Image(sentinel1.filterDate("2022-01-01","2022-01-31").mean()).select(["VV","VH"],["VVjan","VHjan"]);
s1Feb = ee.Image(sentinel1.filterDate("2022-02-01","2022-02-28").mean()).select(["VV","VH"],["VVfeb","VHfeb"])
s1Mar = ee.Image(sentinel1.filterDate("2022-03-01","2022-03-31").mean()).select(["VV","VH"],["VVmar","VHmar"])
s1Apr = ee.Image(sentinel1.filterDate("2022-04-01","2022-04-30").mean()).select(["VV","VH"],["VVapr","VHapr"])

sentinel1mean = s1Dec.addBands(s1Jan)\
                 .addBands(s1Feb)\
                 .addBands(s1Mar)\
                 .addBands(s1Apr)

#print(sentinel1mean.getInfo())

### Speckle filtering 

### Stack Sentinel 1 and Sentinel 2

In [25]:
composite = ee.Image.cat([(s2median),(sentinel1mean)])
sentinel = composite.clip(jedeb)
#print(Sentinel.getInfo())

#Apply reduce the radar speckle by smoothing  
smoothing_radius = 10
#sentinel = sentinel.focal_mean(smoothing_radius, 'circle', 'meters')

## Train the Classifier

In [63]:
bands = ['VVdec', 'VHdec', 'VVjan', 'VHjan', 'VVfeb', 'VHfeb', 'VVmar', 'VHmar', 'VVapr', 'VHapr']
# This property of the table stores the land cover labels.
label = 'Cover'

# Overlay the points on the imagery to get training.
sample = sentinel.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 10}
)

In [71]:
classifier = ee.Classifier.libsvm(**{
  'kernelType': 'RBF',
  'gamma': 0.9,
  'cost': 128
}).train(sample, label, bands)

## Classify the Image

In [72]:
# Classify the image with the same bands used for training.
classified_RF2 = sentinel.select(bands).classify(classifier)

## Accuracy assessment

### Training

In [66]:
train_accuracy = classifier.confusionMatrix()
#train_accuracy.getInfo()

In [30]:
#train_accuracy.accuracy().getInfo()

In [31]:
#train_accuracy.kappa().getInfo()

In [32]:
#train_accuracy.producersAccuracy().getInfo()

In [33]:
#train_accuracy.consumersAccuracy().getInfo()

### Validation

In [73]:
# This property of the table stores the land cover labels.
label = 'Cover'

band = ['classification']
# Overlay the points on the imagery to get training.
test = classified_RF2.select(band).sampleRegions(
    **{'collection': validationGcp, 'properties':[label],'scale': 10}
)

In [74]:
test_accuracy = test.errorMatrix('Cover', 'classification')
#test_accuracy.getInfo()

In [75]:
test_accuracy.accuracy().getInfo()

0.3288888888888889

In [37]:
test_accuracy.kappa().getInfo()

0.029950890817725024

In [38]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [0],
 [1],
 [0.029411764705882353],
 [0.045454545454545456],
 [0],
 [0],
 [0.1111111111111111]]

In [39]:
test_accuracy.consumersAccuracy().getInfo()

[[0, 0, 0.3167420814479638, 1, 1, 0, 0, 1]]

## Improving Classification using Vegetation Indices

### Vegetation Indices 

EVI

In [40]:
def getEVI(image):
    # Compute the EVI using an expression.
    EVI = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'BLUE': image.select('B2')
        }).rename("EVI")

    image = image.addBands(EVI)

    return(image)

evi = sentinel2.map(getEVI).median().clip(jedeb).select('EVI')

print(evi.getInfo())

{'type': 'Image', 'bands': [{'id': 'EVI', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


NDVI

In [41]:
# Calculate NDVI
def getNDVI(image):
    # Compute the EVI using an expression.
    NDVI = image.expression(
        '((NIR - RED) / (NIR + RED ))', {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
        }).rename("NDVI")

    image = image.addBands(NDVI)

    return(image)

ndvi = sentinel2.map(getNDVI).median().clip(jedeb).select('NDVI')

print(ndvi.getInfo())

{'type': 'Image', 'bands': [{'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


NDBI

In [42]:
# Calculate NDBI
def getNDBI(image):
    # Compute the EVI using an expression.
    NDBI = image.expression(
        '((SWIR - NIR) / (SWIR + NIR ))', {
            'SWIR': image.select('B11'),
            'NIR': image.select('B8'),
        }).rename("NDBI")

    image = image.addBands(NDBI)

    return(image)

ndbi = sentinel2.map(getNDBI).median().clip(jedeb).select('NDBI')

print(ndbi.getInfo())

{'type': 'Image', 'bands': [{'id': 'NDBI', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [43]:
composite = ee.Image.cat([(sentinel),(evi),(ndvi),(ndbi)])

sentinel_vi = composite.clip(jedeb)

## Train the classifier 

In [44]:
bands = ['B2', 'B3', 'B4','B8', 'B8A', 'B11', 'B12',\
        'VVdec', 'VHdec', 'VVjan', 'VHjan', 'VVfeb', 'VHfeb', 'VVmar', 'VHmar', 'VVapr', 'VHapr','EVI','NDBI']
# This property of the table stores the land cover labels.
label = 'Cover'

# Overlay the points on the imagery to get training.
sample = sentinel_vi.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 10}
)

In [45]:
classifier = ee.Classifier.libsvm(**{
  'kernelType': 'RBF',
  'gamma': 0.5,
  'cost': 128
}).train(sample, label, bands)

In [46]:
# Classify the image with the same bands used for training.
classified_RF3 = sentinel_vi.select(bands).classify(classifier)

## Accuracy Assessment

### Training

In [47]:
train_accuracy = classifier.confusionMatrix()
#train_accuracy.getInfo()

In [48]:
#train_accuracy.accuracy().getInfo()

In [49]:
#train_accuracy.kappa().getInfo()

In [50]:
#train_accuracy.producersAccuracy().getInfo()

In [51]:
#train_accuracy.consumersAccuracy().getInfo()

### Validation

In [52]:
# This property of the table stores the land cover labels.
label = 'Cover'

band = ['classification']
# Overlay the points on the imagery to get training.
test = classified_RF3.select(band).sampleRegions(
    **{'collection': validationGcp, 'properties':[label],'scale': 10}
)

In [53]:
test_accuracy = test.errorMatrix('Cover', 'classification')
#test_accuracy.getInfo()

In [54]:
test_accuracy.accuracy().getInfo()

0.3288888888888889

In [55]:
test_accuracy.kappa().getInfo()

0.029950890817725024

In [56]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [0],
 [1],
 [0.029411764705882353],
 [0.045454545454545456],
 [0],
 [0],
 [0.1111111111111111]]

In [57]:
test_accuracy.consumersAccuracy().getInfo()

[[0, 0, 0.3167420814479638, 1, 1, 0, 0, 1]]

# Visualize Results

In [58]:
# Define a palette for the classification.
Palette = [
    '419BDF', #water
    '397D49', #trees
    '88B053', #grass
    '6efc6a', #irrigation
    'E49635', #crops
    'C4281B', #built
    'A59B8F', #bare
]
Map.addLayer(classified_RF1, {'palette': Palette, 'min': 1, 'max': 7}, 'classification')
Map

Map(center=[10.41, 37.62], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…

# Calculate Statistics

In [59]:
# Calculate total area of a single class (eq(*))     * is the class number 
total_area = classified_RF1.multiply(ee.Image.pixelArea()).reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': jedeb.geometry(),
  'scale': 10,
  'maxPixels': 1e9
})

total_area.getInfo()

{'classification': 3462370000.1460667}

In [60]:
# Calculate total area of a single class (eq(*))     * is the class number 
area = classified_RF1.eq(4).multiply(ee.Image.pixelArea()).reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': jedeb.geometry(),
  'scale': 10,
  'maxPixels': 1e9
})

area.getInfo()

{'classification': 67787189.58949104}

In [61]:
# Calculate total area of a single class (eq(*))     * is the class number 
area = classified_RF2.eq(4).multiply(ee.Image.pixelArea()).reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': jedeb.geometry(),
  'scale': 10,
  'maxPixels': 1e9
})

area.getInfo()

{'classification': 285450.90710392373}

In [62]:
# Calculate total area of a single class (eq(*))     * is the class number 
area = classified_RF3.eq(4).multiply(ee.Image.pixelArea()).reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': jedeb.geometry(),
  'scale': 10,
  'maxPixels': 1e9
})

area.getInfo()

{'classification': 278903.3173364677}